## Dependencies

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from collections import Counter
import pprint as pp

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('Data/train_clean.csv')
data

,protein_sequence,length,tm
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,341,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,286,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,497,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,265,47.2
4,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,380,48.4
...,...,...,...
23457,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,549,51.8
23458,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,469,37.2
23459,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,128,64.6
23460,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,593,50.7


## Generating FASTA File of All Sequences

In [3]:
fasta_str = ''
sequences = list(data['protein_sequence'])
for i in range(len(sequences)): 
    fasta_str += '>' + str(i) + '\n' + sequences[i] + '\n'
fasta_str = fasta_str[:-1]

file_name = 'Data/fasta_sequences.fasta'
file = open(file_name, 'w')
file.write(fasta_str)
file.close()

## Looking at the FASTA file

Sanity check....

In [4]:
# first few lines of the file
file = open(file_name, 'r')
lines = file.readlines()
pp.pprint(lines[:10])

['>0\n',
 'AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVGMIKDAGDDPDVTHGAEIQAFVRFASEDRLEGGEGVGVVTKPGLGVPVGEPAINPVPRRMIWEAVREVTERPLAVTIAIPGGEELAKKTLNPRLGILGGLSVLGTTGVVKPYSTSAFRMSVVQAVGVARANGLLEIAATTGGKSERFAQRLLPHLPEMAFIEMGDFVGDVLRAARKVGVEVVRVVGMIGKISKMADGKTMTHAAGGEVNLSLLLSLLKEAGASPKALKEAEGAATARRFLEIALEEGLELFFVNLVRLAQEKLQAYIGERPFVSVALTDFDEGRCLAAWPDREVYR\n',
 '>1\n',
 'AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSRLQAQRRAQRVAWEDGDENVGQTVIPAQEEEGIEKPAEVHPTGKIGAKKLRKLEEKQARKAQREAEEAEREERKRLESQREAEWKKEEERLRLKEEQKEEEERKAQEEQARREHEEYLKLKEAFVVEEEGVSETMTEEQSHSFLTEFINYIKKSKVVLLEDLAFQMGLRTQDAINRIQDLLTEGTLTGVIDDRGKFIYITPEELAAVANFIRQRGRVSITELAQASNSLISWGQDLPAQAS\n',
 '>2\n',
 'AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYATLGVAKNANGKDIKKAYYQLAKKYHPDTNKEDPDAGRKFQEVSEAYEVLSDEQKRREYDTYGQTAENIGRQGGGFPGGGAGGFGPEGFSQSWQFRSSIDPEELFRKIFGEGNFRTNSFDDFADSKFGFGQAQEMVMDLTFAQAARGVNKDVNVNVVDQCPKCAGTKCEPGTKPGRCQYCNGTGFETVSTGPFVMRSTCRYCQGTRQHIKYPCSECEGKGRTVQRRKVTVPVPAGIENGQTVRMQVGSKELFVTFRVERSDYFRREGADVHTDAAISLAQAVLGG

In [5]:
# last few lines of the file
pp.pprint(lines[-10:])

['>23457\n',
 'YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQGVETLPPGKVRWPDFNQEAYVGGTMVRSGQDPYARNKFNQVESDKLHMDRGIPDTRHDQCQRKQWRVDLPATSVVITFHNEARSALLRTVVSVLKRSPPHLIKEIILVDDYSNDPEDGALLGKIEKVRVLRNDRREGLMRSRVRGADAAQAKVLTFLDSHCECNERWLEPLLERVAEDRTRVVSPIIDVINMDNFQYVGASADLKGGFDWNLVFKWDYMTPEQRRSRQGNPVAPIKTPMIAGGLFVMDKLYFEELGKYDMMMDVWGGENLEISFRVWQCGGSLEIIPCSRVGHVFRKQHPYTFPGGSGTVFARNTRRAAEVWMDEYKHFYYAAVPSARNVPYGNIQSRLELRKKLGCKPFKWYLDNVYPELRVPDHQDIAFGALQQGTNCLDTLGHFADGVVGIYECHNAGGNQEWALTKEKSVKHMDLCLTVVDRSPGSLIRLQGCRENDSRQKWEQIEGNSKLRHVGSNLCLDSRTAKSGGLSVEVCGPALSQQWKFSLNLQ\n',
 '>23458\n',
 'YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLIVSGVRVVDALNTTCSEGKDQETLETLADLHQTFAHFFQRGAAAERYMSSEDQFNLLVAESEASTRSHHHIGGNAALMADRIAANFPSTEVYLVGPIGPRSQALLHPSVKRTNSTRILKDELHVILEYKQGEILGDWVAPSSSRFITSHDHFSGSMVVMEMFFKAIAQFRPDLVVITGVHLLEFQSKEMRQEKMRLIKRNLLQIPPKVPIHLELGSLADEIFSTDVINKILPYVDSLGINEQELTFLSHIANGPHMEEYPVQAGTVHVHKVVEMLHWLLKTYGRDPTGQIASKTGYRLSRIHFHCLTYHIMVSSGTDWSNLAAGLAAGARIAGRLSCNIGANTMDSELLEIRTPANFVLDKKIEKNYQFEA